**Import required libraries and scripts**

In [3]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

software = '/home/mario/DockM8/software'
protein_file = '/home/mario/holiday/drd3/receptor_protoss_prepared.pdb'
ref_file = '/home/mario/holiday/drd3/crystal_ligand_protoss.sdf'
pocket = 'reference'
protonation = 'pkasolver'
docking_library = '/home/mario/holiday/drd3/merged_actives_decoys.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
clustering_method = 'KMedoids'
rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
parallel = 1
ncpus = int(os.cpu_count()/2)
#Create a temporary folder for all further calculations
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

In [ ]:
if os.path.isfile(protein_file.replace('.pdb', '_pocket.pdb')) == False:
    if pocket == 'reference':
        pocket_definition = GetPocket(ref_file, protein_file, 8)
    elif pocket == 'dogsitescorer':
        pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
if os.path.isfile(w_dir+'/temp/final_library.sdf') == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)

In [ ]:
docking_splitted(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus)

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')


In [ ]:
display(all_poses)

In [ ]:
if parallel == 1:
    for metric in clustering_metrics:
        if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
            cluster_futures(metric, clustering_method, w_dir, protein_file, all_poses, ncpus)
else:
    for metric in clustering_metrics:
        if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
            cluster(metric, clustering_method, w_dir, protein_file, all_poses, ncpus)

In [2]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring, parallel, ncpus)


The folder: /home/mario/holiday/drd3/temp/rescoring_RMSD_clustered already exists

[2023-Mar-24 15:57:27]: /gnina_rescoring folder already exists, skipping gnina rescoring

[2023-Mar-24 15:57:27]: /AD4_rescoring folder already exists, skipping AD4 rescoring

[2023-Mar-24 15:57:27]: /chemplp_rescoring folder already exists, skipping chemplp rescoring

[2023-Mar-24 15:57:27]: /rfscorevs_rescoring folder already exists, skipping rfscorevs rescoring

[2023-Mar-24 15:57:27]: /LinF9_rescoring folder already exists, skipping LinF9 rescoring
The folder: /home/mario/holiday/drd3/temp/rescoring_RMSD_clustered/SCORCH_rescoring/ was created

[2023-Mar-24 15:57:27]: Converting protein file to .pdbqt ...

[2023-Mar-24 15:57:40]: Converting SDF file RMSD_clustered.sdf to .pdbqt files...
Converted 84600 molecules.

[2023-Mar-24 15:58:30]: Rescoring with SCORCH


FileNotFoundError: [Errno 2] No such file or directory: '/home/mario/holiday/drd3/temp/rescoring_RMSD_clustered/SCORCH_rescoring/scoring_results.csv'

In [ ]:
apply_consensus_methods(w_dir, clustering_metrics)
calculate_EFs(w_dir, docking_library)

The folder: /home/mario/holiday/pyrd/temp/ranking already exists
The folder: /home/mario/holiday/pyrd/temp/consensus was created


In [ ]:
apply_consensus_methods_combinations(w_dir, clustering_metrics)
calculate_EFs_combinations(w_dir, docking_library)